In [1]:
from b2d.testbed.backend.jupyter import JupyterGui
import ipywidgets
import ipycanvas
import b2d

# Newtons Cradle
Here we try to implement 
[newtons cradle](https://en.wikipedia.org/wiki/Newton%27s_cradle) in Box2D
The source code for the example is in `plot_newtons_cradle.py`

![Image](https://upload.wikimedia.org/wikipedia/commons/d/d3/Newtons_cradle_animation_book_2.gif)

In [ ]:
from plot_newtons_cradle import NewtonsCradle  as Example
s = JupyterGui.Settings(resolution=[800,300])
Example.run(JupyterGui, gui_settings=s)

<p style=text-align: center>
    <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Angry_Birds_logo_2015.svg/2880px-Angry_Birds_logo_2015.svg.png width=30%>
</p>
The following shows an example simmilar to the game "Angry Birds"
   

In [17]:
from plot_angry_shapes import AngryShapes  as Example
settings = JupyterGui.Settings(fps=30, scale=2, translate=(20,100), resolution=(1000,500))
Example.run(JupyterGui,gui_settings=settings)

Output()

AngryShapes.Settings(substeps=2, n_velocity_steps=1, n_position_iterations=1)


# Gauss Machine
This example shows how to get a gaussian distribution from an uniform one with a mechanical decivce.
Note that the particles have a lifetime of 25 seconds, so the bins will at some point not get fuller (but also your RAM will not get fuller ;) )

In [2]:
from plot_gauss_machine import GaussMachine  as Example
Example.run(JupyterGui)

Output()

TestbedBase.Settings(substeps=1, n_velocity_steps=1, n_position_iterations=1)


# Color Mixing
This example show the color mixing capabilities of liquidfun

In [3]:
from plot_color_mixing import ColorMixing  as Example
Example.run(JupyterGui)

Output()

TestbedBase.Settings(substeps=1, n_velocity_steps=1, n_position_iterations=1)


# Function Shape
This exanmple shows how to draw a chain shape in the shape of a function

In [ ]:
from plot_function_shape import FunctionShape  as Example
Example.run(JupyterGui)

In [ ]:
from plot_goo import Goo  as Example
Example.run(JupyterGui)